# Scratch
Old code

## Baseline Model
Training before Refactoring

In [ ]:
# class ISICDataset(Dataset):
#     def __init__(self, df, img_dir, transform=None):
#         self.df = df.reset_index(drop=True)
#         self.img_dir = img_dir
#         self.transform = transform
#         self.labels = self.df.iloc[:, 1:].values.argmax(axis=1)
#         self.image_ids = self.df.iloc[:, 0].values

#     def __len__(self):
#         return len(self.image_ids)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.image_ids[idx] + ".jpg")
#         image = Image.open(img_path).convert("RGB")
#         label = self.labels[idx]
#         if self.transform:
#             image = self.transform(image)
#         return image, label

In [ ]:
# # OLD CODE

# # train
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# model.train()
# for images, labels in train_loader:
#     images, labels = images.to(device), labels.to(device)
#     optimizer.zero_grad()
#     outputs = model(images)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     optimizer.step()
#     print(f"Loss: {loss.item():.4f}")


# # eval
# model.eval()
# correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in val_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         preds = torch.argmax(outputs, dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

# accuracy = correct / total
# print(f"Validation Accuracy: {accuracy:.2%}")

In [ ]:
# Class-weighted loss
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df["label"]),
    y=train_df["label"].values
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f"\nCUDA available: {torch.cuda.is_available()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Epoch {epoch+1} [Train]:")

    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.long().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    avg_train_loss = running_loss / len(train_loader.dataset)
    print(f"\nEpoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")

    # Validation
    print(f"Epoch {epoch+1} [Val]:")
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.long().to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_acc = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=[index_to_label[i] for i in range(8)]))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

## Fusion model
Training loop before refactoring

In [ ]:
# from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

# # Load pretrained EfficientNet-B3 with weights
# weights = EfficientNet_B3_Weights.IMAGENET1K_V1
# model = efficientnet_b3(weights=weights)

# # Replace classifier head for 8-class output
# model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 8)

# # Move to device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# with open("/content/drive/My Drive/midas/models/baseline.py", "w") as f:
#   f.write("# models/baseline_model.py")

from models.baseline_model import BaselineModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaselineModel(num_classes=8).to(device)

In [ ]:
#########
# Setup #
#########

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# prep to use class-weighted loss
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df["label"]),
    y=train_df["label"].values
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# get metadata dim from a sample
_, meta_sample, _ = next(iter(train_loader))
meta_dim = meta_sample.shape[1]
# initialize model
model = FusionModel(num_metadata_features=meta_dim, num_classes=8).to(device)
# loss and optimizer
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
train_losses = []
val_losses = []

#################
# TRAINING LOOP #
#################

num_epochs = 5
for epoch in range(num_epochs):
    print("CUDA available:", torch.cuda.is_available())
    print("Current device:", torch.cuda.current_device())
    model.train()
    train_loss = 0.0
    try: # flag any errors that arise during trainig
        for imgs, metas, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.long().to(device)
            optimizer.zero_grad()
            outputs = model(imgs, metas)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * imgs.size(0)
            batch_loss = loss.item()
            if not np.isfinite(batch_loss):
                print(f"[Error] Non-finite loss: {batch_loss} at epoch {epoch}, batch {i}")
                break
    except Exception as e:
        print(f"Exception during training: {e}")
        traceback.print_exc()
        break

    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    print(f"\nEpoch {epoch+1}: Train Loss = {train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.long().to(device)
            outputs = model(imgs, metas)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    val_acc = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")

    # Per-class metrics
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=[index_to_label[i] for i in range(8)]))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))